# Basic Tensorflow based NeuralNet without any libs
This python notebook is for studying how to make NeuralNet model using Tensorflow
Keras is just used for load the dataset
Reference:
1. https://www.kaggle.com/code/enriqueabad/using-tensorflow-from-scratch-without-keras/notebook
2. https://www.tensorflow.org/tutorials/quickstart/beginner?hl=ko

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import os

from matplotlib import pyplot as plt
from time import time
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder, scale
from sklearn.model_selection import train_test_split
from tqdm import tqdm
print("TensorFlow Version: ", tf.__version__)

TensorFlow Version:  2.10.0


In [2]:
"""
Load MNist Dataset from the keras
"""
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
"""Normalize the input to make it inside of range 0 to 1"""
x_train, x_test = x_train / 255.0, x_test / 255.0
print(type(x_train))
print(x_train.shape)
print(y_train.reshape(-1,1).shape)
print(y_test.reshape(-1,1).shape)

<class 'numpy.ndarray'>
(60000, 28, 28)
(60000, 1)
(10000, 1)


In [4]:
"""
Make label as an one hot vector
"""
label_encoder = OneHotEncoder(categories=[np.arange(10)])
"""
Need to make 1D array to 2D array So use ndarray.reshape()
We use -1 to indicate the length of row or column dynamically
"""
y_train = label_encoder.fit_transform(y_train.reshape(-1,1))
y_test = label_encoder.fit_transform(y_test.reshape(-1,1))

In [5]:
print(y_train.shape)

(60000, 10)


# 1. Neural Network without hidden layers

In [6]:
"""
Make weight tensor
We have 784 pixel image so the weight tensor size should be 784 x 10
-> 784 pixel and out put is 10
tf.Variable => tensor in tensorflow is immutable.
So we use variables which is tensor whose value can be changed by running ops on it. 
"""
# weight_random = tf.Variable(tf.random.uniform([784,10]))
weight_random = tf.Variable(tf.zeros([784,10]))

In [31]:
"""
Make the loss function -> mean squared error loss and cross-entropy loss 
"""
def loss_MSE(target_y, predicted_y):
    return tf.reduce_mean(tf.square(target_y-predicted_y))

def loss_CrossEntrophy(target_y, predicted_y):
    return -tf.reduce_sum(tf.reduce_mean(np.multiply(target_y,tf.math.log(predicted_y+1e-12)),axis=0))

In [17]:
"""Define Flatten Layer"""
def flatten(t):
    """
    By using np.reshape, reshape tensor to 1D
    """
    return t.reshape(t.shape[0],-1)

In [18]:
print(flatten(x_train).shape)

(60000, 784)


## Make model
Make model using tf.Module makes it easier to build own model
tf.Module is Base neural network module class
Module  is an named container for tf.Variable s, other tf.Module s and functions which apply to user input.

In [10]:
class Model1(tf.Module):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        """
        Weight starts with zeros 
        But it is better to start with random

        """
        self.w = tf.Variable(tf.zeros([784,10]))
        #self.w = tf.Variable(tf.random.uniform[784,10])
        self.b = tf.Variable(0.0)
    
    def __call__(self, x):
        """ 
        With just call the class, we can predict
        y_predicted = Model1()(x_data)
        """
        x = flatten(x)
        return tf.nn.softmax((x @ self.w + self.b))


In [11]:
model = Model1()

In [12]:
def train(model,x,y,learning_rate):
    with tf.GradientTape() as t:
        current_loss = loss_CrossEntrophy(y,model(x))
    
    dw, db = t.gradient(current_loss, [model.w, model.b])

    model.w.assign_sub(learning_rate*dw)
    model.b.assign_sub(learning_rate*db)

In [32]:
for i_epochs in tqdm(range(50)):
    train(model, x_train, y_train, learning_rate=0.2)
    train_loss = loss_CrossEntrophy(y_train, model(x_train))
    test_loss = loss_CrossEntrophy(y_test, model(x_test))
    print(f"Training loss in epoch {i_epochs} = {train_loss.numpy()}. Test loss = {test_loss.numpy()}")

  0%|          | 0/50 [00:00<?, ?it/s]